In [1]:
import os
import sys
import random
import time
from random import seed, randint
import argparse
import platform
from datetime import datetime
import imp
import numpy as np
import fileinput
from itertools import product
import pandas as pd
from scipy.interpolate import griddata
from scipy.interpolate import interp2d
import seaborn as sns
from os import listdir

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.interpolate import griddata
import matplotlib as mpl
sys.path.insert(0,'..')
# from notebookFunctions import *
# from .. import notebookFunctions

%matplotlib inline
plt.rcParams['figure.figsize'] = (10,6.180)    #golden ratio
# %matplotlib notebook
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '/Users/weilu/openmmawsem/')

from openmmawsem import *

In [2]:
input_pdb_filename = "/Users/weilu/Research/server/dec_2018/T0958_single_chain/openmm_T0958/T0958-openmmawsem.pdb"

In [3]:
data = pd.read_table(input_pdb_filename, sep="\s+", header=None, names=["_","i","type","res","chain","res_id", "x","y","z","_1","_2","_3"])
data = data.dropna().reset_index()
data["res_id"] = data["res_id"].apply(lambda x: int(x))



In [7]:
os.chdir("/Users/weilu/Research/server/dec_2018/T0958_single_chain/openmm_T0958")

In [11]:
def read_beta_parameters():
    ### directly copied from Nick Schafer's
    #os.chdir(parameter_directory)
    in_anti_HB = open("anti_HB", 'r').readlines()
    in_anti_NHB = open("anti_NHB", 'r').readlines()
    in_para_HB = open("para_HB", 'r').readlines()
    in_para_one = open("para_one", 'r').readlines()
    in_anti_one = open("anti_one", 'r').readlines()

    p_par = np.zeros((20))
    p_anti = np.zeros((20))
    p_antihb = np.zeros((20,20,2))
    p_antinhb = np.zeros((20,20,2))
    p_parhb = np.zeros((20,20,2))

    for i in range(20):
        p_par[i] = float(in_para_one[i].strip())
        p_anti[i] = float(in_anti_one[i].strip())
        for j in range(20):
            p_antihb[i][j][0] = float(in_anti_HB[i].strip().split()[j])
            p_antinhb[i][j][0] = float(in_anti_NHB[i].strip().split()[j])
            p_parhb[i][j][0] = float(in_para_HB[i].strip().split()[j])

    for i in range(20):
        for j in range(20):
            p_antihb[i][j][1] = float(in_anti_HB[i+21].strip().split()[j])
            p_antinhb[i][j][1] = float(in_anti_NHB[i+21].strip().split()[j])
            p_parhb[i][j][1] = float(in_para_HB[i+21].strip().split()[j])
    return p_par, p_anti, p_antihb, p_antinhb, p_parhb

In [12]:
p_par, p_anti, p_antihb, p_antinhb, p_parhb = read_beta_parameters()

In [10]:
p_par

array([ 0.04594139, -1.14228296, -0.98018169, -0.95481044,  0.06045979,
       -0.94054347, -0.94427794, -0.32640567, -0.51735067,  0.67487478,
        0.55926394, -1.37892628,  0.31142619,  0.22928254, -0.54442596,
       -0.59238559, -0.70182824, -0.3375912 , -0.41496691,  0.66063666])

In [13]:
test = np.loadtxt("anti_HB")

In [17]:
p_antihb

array([[[ 0.     ,  0.     ],
        [-1.8013 ,  0.     ],
        [ 0.     ,  0.     ],
        [ 0.     ,  0.     ],
        [ 0.     ,  0.     ],
        [ 0.     ,  0.     ],
        [ 0.     , -0.41122],
        [ 0.88204,  0.     ],
        [ 0.     ,  0.     ],
        [ 0.     ,  0.32913],
        [ 0.     ,  0.24986],
        [ 0.     ,  0.     ],
        [ 0.     ,  0.     ],
        [ 0.     ,  0.     ],
        [ 0.     ,  0.     ],
        [ 0.     , -0.35787],
        [ 0.     ,  0.     ],
        [ 0.67395,  0.     ],
        [ 0.     ,  0.     ],
        [ 0.     ,  0.20977]],

       [[ 0.     , -0.45514],
        [ 0.     ,  0.     ],
        [ 0.     ,  0.     ],
        [ 0.     ,  1.00415],
        [ 0.     ,  0.     ],
        [ 0.     ,  0.96345],
        [ 1.19422,  0.92262],
        [ 0.     ,  0.     ],
        [ 0.82412,  0.     ],
        [ 0.     , -0.39318],
        [ 0.     , -0.37451],
        [ 0.     ,  0.     ],
        [ 0.     ,  0.     ],
        

In [16]:
p_antinhb.shape

(20, 20, 2)

In [14]:
test

array([[ 0.     , -1.8013 ,  0.     ,  0.     ,  0.     ,  0.     ,
         0.     ,  0.88204,  0.     ,  0.     ,  0.     ,  0.     ,
         0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.67395,
         0.     ,  0.     ],
       [ 0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,
         1.19422,  0.     ,  0.82412,  0.     ,  0.     ,  0.     ,
         0.     ,  0.     ,  0.     ,  0.     ,  0.50158,  0.     ,
         0.     ,  0.     ],
       [ 0.     ,  0.     ,  0.89208,  0.     ,  0.     ,  0.99199,
         0.67662,  0.     ,  0.     ,  0.     , -1.13044,  0.     ,
         1.02639,  0.     ,  0.     ,  0.98118,  0.     ,  0.     ,
         0.     ,  0.     ],
       [ 0.     ,  1.26022,  0.     ,  1.24896,  0.     ,  0.     ,
         0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  1.31277,
         0.     ,  0.     ,  0.     ,  0.95689,  0.     ,  0.     ,
         0.     ,  0.     ],
       [ 0.     ,  0.     ,  0.     ,  0.     ,  1.14724,  0.     ,


In [6]:
data

,index,_,i,type,res,chain,res_id,x,y,z,_1,_2,_3
0,0,ATOM,1.0,CA,NGP,A,1,29.552,32.708,45.263,1.0,0.0,C
1,1,ATOM,2.0,C,NGP,A,1,29.201,33.000,43.806,1.0,0.0,C
2,2,ATOM,3.0,O,NGP,A,1,28.185,33.568,43.494,1.0,0.0,O
3,3,ATOM,4.0,CB,NGP,A,1,29.753,31.202,45.547,1.0,0.0,B
4,4,ATOM,5.0,N,NGP,A,2,30.070,32.597,42.936,1.0,0.0,N
5,5,ATOM,10.0,H,NGP,A,2,30.890,32.141,43.190,1.0,0.0,H
6,6,ATOM,6.0,CA,NGP,A,2,29.925,32.777,41.484,1.0,0.0,C
7,7,ATOM,7.0,C,NGP,A,2,28.586,32.302,40.892,1.0,0.0,C
8,8,ATOM,8.0,O,NGP,A,2,27.794,33.049,40.399,1.0,0.0,O
9,9,ATOM,9.0,CB,NGP,A,2,31.105,32.190,40.705,1.0,0.0,B


In [5]:
compute_chi(data)

IndexError: index 78 is out of bounds for axis 0 with size 78

In [4]:
def compute_chi(data):
    ca_all = data.query("type == 'CA'")[["x","y","z"]].values
    cb_all = data.query("type == 'CB'")[["x","y","z"]].values
    c_all = data.query("type == 'C'")[["x","y","z"]].values
    n_all = data.query("type == 'N'")[["x","y","z"]].values
    energy = 0 
    for i in range(len(n_all)):
        ca = ca_all[i]
        cb = cb_all[i]
        c = c_all[i]
        n = n_all[i]
        chi0 = -0.71
        k_chi = 60*4.184
        r_ca_cb = cb-ca
        r_c_ca = ca-c
        r_ca_n = n-ca
        norm_r_ca_cb = np.sum(r_ca_cb**2)**0.5
        norm_r_c_ca = np.sum(r_c_ca**2)**0.5
        norm_r_ca_n = np.sum(r_ca_n**2)**0.5
        a = np.cross(-r_c_ca,r_ca_n)/norm_r_c_ca/norm_r_ca_n
        chi = np.dot(a,r_ca_cb)/norm_r_ca_cb
        dchi = chi - chi0
        energy += k_chi*dchi*dchi
    return energy